In [14]:
acc_covmis_before = 0.9885245901639345
f1_covmis_before = 0.9843518094365552

acc_covmis_after = 0.9881862404447533
f1_covmis_after = 0.9827856884000605

(acc_covmis_before - acc_covmis_after), (f1_covmis_before - f1_covmis_after)


(0.0003383497191811191, 0.001566121036494672)

In [16]:
acc_liar2_before = 0.9378068739770867
f1_liar2_before = 0.9074319933338915

acc_liar2_after = 0.9324912891986062
f1_liar2_after = 0.9217986228840329

(acc_liar2_before - acc_liar2_after), (f1_liar2_before - f1_liar2_after)


(0.005315584778480487, -0.014366629550141452)

In [2]:
acc = 0.9885245901639345
human_baseline = 0.9254098360655738

(acc - human_baseline) * 100


6.311475409836065

In [59]:
136 * 6 / 12192

0.06692913385826772

In [71]:
0.067 * 12192

816.864

In [74]:
(136 * 6 + 5) / 12192

0.06733923884514435

In [61]:
import numpy as np
from statsmodels.stats.inter_rater import fleiss_kappa

# 构建标注矩阵
data = np.array(
    [[2, 1], [1, 2], [2, 1], [2, 1], [2, 1], [2, 1]] * 136 + [[2, 1], [1, 2], [2, 1], [2, 1], [2, 1]] + [
    [3, 0],  # 三位标注者均选择第一类
    [3, 0],
    [3, 0],
    [3, 0],
    [0, 3],
    [3, 0],  # 三位标注者均选择第一类
]* 1895 + [[3, 0]]) 

# 计算 Fleiss' Kappa
kappa = fleiss_kappa(data, method='fleiss')
print("Fleiss' Kappa:", kappa)


Fleiss' Kappa: 0.8489833376129666


In [75]:
len(data)


12192

Existing misinformation detection benchmark datasets (e.g., COVMIS and LIAR2) are limited by their reliance on fact-checking labels that are prone to fac-tual inaccuracies due to cognitive con-straints of fact-checkers and outdated la-bels. Prior real-time misinformation de-tection tasks have been hindered by the dual problems of label redundancy and cold start. To this end, we propose a nov-el Cyclic Evidence-based Misinfor-mation Detection (CEMD) framework, which incorporates two core mechanisms: (i) a Retrieval Augmented Generation (RAG) pipeline that accesses the latest external knowledge to augment insuffi-cient prior knowledge, and (ii) a cyclic evidence-bootstrapping mechanism that mitigates label redundancy and cold start. We introduce an improved bench-mark dataset, COVMIS2, built upon COVMIS, and conduct comprehensive experiments to evaluate the efficacy of our framework. Our results demonstrate that CEMD outperforms the prior state-of-the-art (SOTA) baseline on LIAR2 by 11.95% and surpasses the human baseline on COVMIS2 by 6.31%, leveraging the Llama-3-70B-Instruct model to augment prior knowledge and the DoRA fine-tuned Llama-3-8B-Instruct model for bi-nary classification.

Existing misinformation detection benchmark datasets (e.g., COVMIS and LIAR2) are limited by their reliance on fact-checking labels that are prone to factual inaccuracies due to outdated la-bels and cognitive constraints of fact-checkers. Moreover, prior real-time mis-information detection tasks have been hindered by the dual problems of label redundancy and cold start. To this end, we propose a novel Cyclic Evidence-based Misinformation Detection (CEMD) framework, which incorporates two core mechanisms: (i) a Retrieval Augmented Generation (RAG) pipeline that accesses the latest external knowledge to augment insufficient prior knowledge, and (ii) a cyclic evidence-bootstrapping mecha-nism that mitigates label redundancy and cold start. We introduce an improved benchmark dataset, COVMIS2, built up-on COVMIS, and conduct comprehensive experiments to evaluate the efficacy of our framework. Our results demonstrate that CEMD outperforms the prior state-of-the-art (SOTA) baseline on LIAR2 by 11.95% and surpasses the human baseline on COVMIS2 by 6.31%, leveraging the Llama-3-70B-Instruct model to augment prior knowledge and the DoRA fine-tuned Llama-3-8B-Instruct model for bi-nary classification.


In [1]:
import torch
import torch.utils.benchmark as benchmark
from torch.backends.cuda import sdp_kernel, SDPBackend
# from torch.nn.attention import sdp_kernel
import torch.nn.functional as F
import os
import numpy as np
import random

DEFAULT_SEED = 42

def set_seed(seed=DEFAULT_SEED):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True


# os.environ['CUDA_VISIBLE_DEVICES'] = '2'
# Lets define a helpful benchmarking function:
def benchmark_torch_function_in_microseconds(f, *args, **kwargs):
    t0 = benchmark.Timer(
        stmt="f(*args, **kwargs)", globals={"args": args, "kwargs": kwargs, "f": f}
    )
    return t0.blocked_autorange().mean * 1e6

# Lets define the hyper-parameters of our input
batch_size = 32
max_sequence_len = 1024
num_heads = 32
embed_dimension = 32

dtype = torch.float16
device = "cuda:0"

query = torch.rand(batch_size, num_heads, max_sequence_len, embed_dimension, device=device, dtype=dtype)
key = torch.rand(batch_size, num_heads, max_sequence_len, embed_dimension, device=device, dtype=dtype)
value = torch.rand(batch_size, num_heads, max_sequence_len, embed_dimension, device=device, dtype=dtype)

print(f"The default implementation runs in {benchmark_torch_function_in_microseconds(F.scaled_dot_product_attention, query, key, value):.3f} microseconds")

# Lets explore the speed of each of the 3 implementations

# Helpful arg mapper
backend_map = {
    SDPBackend.MATH: {"enable_math": True, "enable_flash": False, "enable_mem_efficient": False},
    SDPBackend.FLASH_ATTENTION: {"enable_math": False, "enable_flash": True, "enable_mem_efficient": False},
    SDPBackend.EFFICIENT_ATTENTION: {
        "enable_math": False, "enable_flash": False, "enable_mem_efficient": True}
}

with sdp_kernel(**backend_map[SDPBackend.MATH]):
    print(f"The math implementation runs in {benchmark_torch_function_in_microseconds(F.scaled_dot_product_attention, query, key, value):.3f} microseconds")

with sdp_kernel(**backend_map[SDPBackend.FLASH_ATTENTION]):
    try:
        print(f"The flash attention implementation runs in {benchmark_torch_function_in_microseconds(F.scaled_dot_product_attention, query, key, value):.3f} microseconds")
    except RuntimeError:
        print("FlashAttention is not supported. See warnings for reasons.")

with sdp_kernel(**backend_map[SDPBackend.EFFICIENT_ATTENTION]):
    try:
        print(f"The memory efficient implementation runs in {benchmark_torch_function_in_microseconds(F.scaled_dot_product_attention, query, key, value):.3f} microseconds")
    except RuntimeError:
        print("EfficientAttention is not supported. See warnings for reasons.")


The default implementation runs in 927.106 microseconds


/home/hanlv/miniconda3/envs/swift/lib/python3.11/site-packages/torch/backends/cuda/__init__.py:342: FutureWarning: torch.backends.cuda.sdp_kernel() is deprecated. In the future, this context manager will be removed. Please see, torch.nn.attention.sdpa_kernel() for the new context manager, with updated signature.
  warnings.warn(


The math implementation runs in 10094.952 microseconds
The flash attention implementation runs in 918.287 microseconds
The memory efficient implementation runs in 1782.254 microseconds


In [1]:
dtype = torch.bfloat16

set_seed()
query = torch.rand(batch_size, num_heads, max_sequence_len, embed_dimension, device=device, dtype=dtype)
key = torch.rand(batch_size, num_heads, max_sequence_len, embed_dimension, device=device, dtype=dtype)
value = torch.rand(batch_size, num_heads, max_sequence_len, embed_dimension, device=device, dtype=dtype)


# with sdp_kernel(**backend_map[SDPBackend.FLASH_ATTENTION]):
# print(F.scaled_dot_product_attention(query, key, value))
    


In [2]:
from decimal import Decimal
import decimal

def data_round(x):
    return str(Decimal(x).quantize(Decimal('0.0000'), rounding=decimal.ROUND_HALF_UP))


def res_trans(res):
    print(data_round(res["ACC"]))
    print(data_round(res["F1"]))
    print(data_round(res["Precision"]))
    print(data_round(res["Recall"]))

res = {
        "model": "Mistral-7B-Instruct-v0.3",
        "train_test_split": "8:2",
        "train_ratio": "1.0",
        "train_loss": 0.06997728,
        "lr": "1.4e-4",
        "ACC": 0.9877049180327869,
        "F1": 0.9832533980502562,
        "Precision": 0.9883868032359371,
        "Recall": 0.9783695652173913
    }

res_trans(res)


0.9877
0.9833
0.9884
0.9784


In [4]:
import numpy as np
x = np.array([0.9869,	0.9877,	0.9852,	0.9877,	0.9877])
x.mean()
# 0.98672 0.98622 0.9852399999999999

0.98704

In [10]:
import os
file = '/home/hanlv/workspace/code/research/infodemic/LLM/swift/examples/pytorch/llm/output/liar2/Llama-3-8B-Instruct/with_llama3_info/brave/data1a-split=8:1:1-ratio=1.0/dora-r=8/lr=1.1e-4-20240827-13:17:06'



get_ckpts(file)

['checkpoint-1148']